<a href="https://colab.research.google.com/github/a-afonso/03MAIR-Algoritmos-de-optimizacion/blob/master/SEMINARIO/SEMINARIO_Augusto_Afonso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
<b>Nombre y Apellidos: </b>Augusto Afonso Losada <br>
<b>Url: </b>https://github.com/a-afonso/03MAIR-Algoritmos-de-optimizacion/tree/master/SEMINARIO<br>
  <b>Problema:</b>
> 1. Elección de grupos de población homogéneos <br>

<b>Descripción del problema: </b>Una productora ganadera nos encarga la tarea de seleccionar grupos de terneros para aplicar 3 tratamientos diferentes. Para cada uno de los tratamientos debemos seleccionar 3 grupos de terneros que sean lo mas homogéneos posible en peso para que en los resultados del tratamiento influya lo menos posible el peso del animal. Disponemos de una población de N animales entre machos y hembras.<br>
<br>
Se solicita diseñar un algoritmo para conseguir una agrupación que cumpla de la mejor manera posible las especificaciones de la productora.<br>
<br>

(*) La respuesta es obligatoria





                                        

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?

¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?





**Respuesta**<br>

Para calcular las posibilidades sin restricciones, debemos suponer que son N animales (independientemente del sexo) y que el peso no influye en la solución.<br>

En estas circunstancias, tenemos 3 tratamientos x 3 grupos = 9 grupos efectivos, por lo cual lo que tenemos son combinaciones de N elementos tomados de 9 en 9 y sin repetición, es decir:<br>

*   N! / 9! (N-9)!

<br>
En el caso con restricciones, aunque podría parecer que la solución puede ser única (la que proponga la combinación más homogénea), en realidad no es posible determinar con exactitud el número de posibles soluciones, ya que por ejemplo, puede haber animales del mismo sexo con el mismo peso, en cuyo caso podrían caer en cualquiera de los grupos efectivos del propio sexo indistintamente.<br>

La peor de las situaciones sería aquella en la que todos los animales tienen el mismo peso y el sexo está divido exactamente al 50%, porque cualquier combinación sería posible, con lo cual las posibilidades coincidirían con el caso anterior:<br>

*   N! / 9! (N-9)!

Paradójicamente, en este último caso cualquier combinación sería la solución óptima, ya que cumpliría con el criterio solicitado.<br>

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Arguméntalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguméntalo).


**Respuesta**

Es posible utilizar una lista de listas para contener la estructura de datos de entrada. La lista contendría N elementos de tipo lista. En esta última estructura, habría tres valores: el identificador del animal, el peso y el sexo.<br>

La estructura de datos devuelta sería una lista de 9 elementos, tantos como grupos efectivos de cada sexo, que a su vez serían listas. Estas últimas tendrían una estructura en la cual sus elementos serían los identificadores de los animales de cada uno de los grupos efectivos.

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

**Respuesta**

Para responder a estas preguntas, habría que definir el concepto de *"Homogeneidad de un Grupo Efectivo de animales"*.<br>

*   Homogeneidad de un Grupo Efectivo de animales (HGE): Peso medio de los animales que componen un grupo efectivo de animales. Donde un grupo efectivo de animales (GE) equivale al conjunto de animales en cada uno de los conjuntos formados por el cómputo de 3 tratamientos x 3 grupos. Es decir hay 9 grupos efectivos (por cada sexo) con sus animales.

Para un determinado GE, habría que calcular su HGE, comparándola con lo desviada que está respecto al peso medio de toda la muestra. La combinación óptima será aquella en la que dicha desviación de todos los GE con respecto al peso medio, es mínima.<br>

Por lo tanto es un problema de minimización. 



Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [6]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
from itertools import permutations

# Inicialización de variables globales
nAnimals = 15
nTreats = 1
nGroups = 2

# Creación de un dataset de muestra
MUESTRA = pd.DataFrame()
MUESTRA["Id"] = [i for i in range(nAnimals)]
MUESTRA["Peso"] = [random.gauss(37, 2.1) for i in range(nAnimals)]
MUESTRA["Sexo"] = [random.getrandbits(1) for i in range(nAnimals)]

# Dividimos el dataset en dos listas, una para cada sexo
MALE = []
FEMALE = []

for index, row in MUESTRA.iterrows():
  if row["Sexo"] == 0:
    MALE.append([int(row["Id"]), row["Peso"], 0])
  else:
    FEMALE.append([int(row["Id"]), row["Peso"], 0])

# Calcula el número máximo de animales por grupo
nMale = len(MALE)
nFemale = len(FEMALE)

nAnimalsGroup = min(nMale, nFemale) // (nTreats * nGroups)


# Función que calcula la homgeneidad de una permutación 
# dados un peso medio y un grupo efectivo de animales
def get_homogeneidad(PERMUTACION, p_medio, nTotalGroups):
  T1 = []
  ix = 0
  for g in range(nTotalGroups):
    T2 = []
    ix = ix + nAnimalsGroup
    for i in range(ix,ix+nAnimalsGroup):
      T2.append(L[ix])
    print(T2)
    T1.append(T2)
  print(T1)
  T0.append(T1)
    


# Función para crear agrupaciones de cualquier sexo
def group_animals(L, nTreats, nGroups, nAnimalsGroup):
  
  RES = []
  
  # Calcula el peso medio
  p_medio = np.mean(L, axis=0)[1]
  
  # Calcula el número total de grupos
  nTotalGroups = nTreats * nGroups
  
  # Calcula el número de animales de la lista
  nAnimalsSex = nTotalGroups * nAnimalsGroup
  
  # Ordenamos la muestra por diferencias con respecto a la media
  # para excluir los más distantes
  print(p_medio)
  print(L)
  for a in L:
    a[2] = abs(p_medio - a[1])
  L = sorted(L, key=lambda x: x[2])
  print(L)
  L = L[0:nAnimalsSex]
  print(L)
  
  # Calcula el nuevo peso medio
  p_medio = np.mean(L, axis=0)[1]
  print(p_medio)
  
  # Calcula las permutaciones de animales
  PERMUTACIONES = list(permutations([L[i][0] for i in range(0, len(L))]))
  PESOS = []
  #
  for PERM in PERMUTACIONES:
    P = []
    for p in PERM:
      P.append(MUESTRA.loc[MUESTRA["Id"] == p, "Peso"].iloc[0])
    PESOS.append(P)
    
    
     
  print(PERMUTACIONES)
  print(PESOS) 
  
  
  
    
  return []
  
R = group_animals(MALE, nTreats, nGroups, nAnimalsGroup)  
print()
R = group_animals(FEMALE, nTreats, nGroups, nAnimalsGroup)


37.4260484549993
[[0, 34.1858147404022, 0], [1, 36.473956237121385, 0], [3, 37.20901045582455, 0], [4, 37.645315771734325, 0], [5, 35.71791676368898, 0], [7, 38.81890159260209, 0], [9, 34.81619469266365, 0], [11, 40.38194343982873, 0], [12, 41.58538240112777, 0]]
[[3, 37.20901045582455, 0.21703799917474953], [4, 37.645315771734325, 0.21926731673502786], [1, 36.473956237121385, 0.9520922178779117], [7, 38.81890159260209, 1.392853137602792], [5, 35.71791676368898, 1.7081316913103137], [9, 34.81619469266365, 2.6098537623356464], [11, 40.38194343982873, 2.9558949848294347], [0, 34.1858147404022, 3.240233714597096], [12, 41.58538240112777, 4.15933394612847]]
[[3, 37.20901045582455, 0.21703799917474953], [4, 37.645315771734325, 0.21926731673502786], [1, 36.473956237121385, 0.9520922178779117], [7, 38.81890159260209, 1.392853137602792], [5, 35.71791676368898, 1.7081316913103137], [9, 34.81619469266365, 2.6098537623356464]]
36.78021591893916
[(3, 4, 1, 7, 5, 9), (3, 4, 1, 7, 9, 5), (3, 4, 1, 5

In [20]:
def get_homogeneidad(PERMUTACION, PESOS, p_medio, nTotalGroups):
  T1 = []
  ix = 0
  s = 0
  for g in range(nTotalGroups):
    T2 = []
    for i in range(ix, ix+nAnimalsGroup):
      T2.append(PESOS[i])
    T1.append(T2)
    ix = ix + nAnimalsGroup
    
    m = np.mean(T2)
    s = s + abs(p_medio - m)
  
  print(T1)
  
  return(s / nTotalGroups)
  
  
get_homogeneidad([0, 2, 4, 1, 3, 5], [35, 37, 39, 36, 38, 40], 37.0, 2)


[[35, 37, 39], [36, 38, 40]]


0.5

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

(*)Calcula la complejidad del algoritmo 

Respuesta

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [0]:
import pandas as pd
import random

# Inicialización de variables globales
nAnimals = 3000
nTreats = 3
nGroups = 3

# Creación de un dataset de muestra
MUESTRA = pd.DataFrame()
MUESTRA["Id"] = [i for i in range(nAnimals)]
MUESTRA["Peso"] = [random.gauss(37, 2.1) for i in range(nAnimals)]
MUESTRA["Sexo"] = [random.getrandbits(1) for i in range(nAnimals)]

Aplica el algoritmo al juego de datos generado

Respuesta

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta